In [88]:
import pandas as pd
import csv
from Plan import CreateSamples_renewed
import importlib
import numpy as np

In [89]:
importlib.reload(CreateSamples_renewed)

<module 'Plan.CreateSamples_renewed' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Plan\\CreateSamples_renewed.py'>

In [175]:
path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\New_Testing_Protocol_Generalized.csv"
chem_path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Chemical Database.csv"

In [176]:
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)
# so lets change this 
# new columns: total_sample_amount, total_sample_unit, component_identity/name, component_unit
# the only issue I could see is nothing since its not adding another row, just using the same axis 0 space provided
# now can keep the column names gnereal like Component 1 Concnetration wtf (keep the enumberating number for now just in case)
#  the only thing is that each component would need to have a unit and name and stuff so it could get full but thats fine
plan['Sample Amount']

0.8

In [194]:
def concentration_from_linspace_all_info(plan, unity_filter = False): # if you go this route you can do whole dataframe operation you just need to verify all component units of the same type
    """ Uses linspaces to create a mesh of component concentrations
    """
    component_linspaces = plan['Component Concentration Linspaces [min, max, n]']
    component_spacing_type = 'linear'

    conc_range_list = [] 
    for conc_linspace in component_linspaces:
        if component_spacing_type == "linear": 
            conc_range_list.append(np.linspace(*conc_linspace))
    conc_grid = np.meshgrid(*conc_range_list)
    
    total_sample_amount = plan['Sample Amount']
    total_sample_amount_unit = plan['Sample Unit']
    component_names = plan['Component Shorthand Names']
    component_units = plan['Component Concentration Units']
    
    data = []
    columns = []
    for component_index in range(len(conc_grid)): 
        n = len(conc_grid[component_index].ravel())
        
        component_name_entry = [component_names[component_index]]*n
        columns.append('Component ' + str(component_index) + ' Name')
        data.append(component_name_entry)
        
        component_unit_entry = [component_units[component_index]]*n
        columns.append('Component ' + str(component_index) + ' Concentration Unit')
        data.append(component_unit_entry)
        
        component_concentration_column = 'Component ' + str(component_index) + ' Concentration Value'
        component_concetration_values = conc_grid[component_index].ravel()
        columns.append(component_concentration_column)
        data.append(component_concetration_values)
        
    component_conc_df = pd.DataFrame(data, columns).T # will terminate here if not needed unity
        
    if unity_filter == True: # generalize this  and make into callable function
        final_component_index = component_index + 1 
        
        component_name_entry = [component_names[final_component_index]]*n
        columns.append('Component ' + str(final_component_index) + ' Name')
        data.append(component_name_entry)
        
        component_unit_entry = [component_units[final_component_index]]*n
        columns.append('Component ' + str(final_component_index) + ' Concentration Unit')
        data.append(component_unit_entry)
        
        concentration_values_isolated = component_conc_df[[col for col in component_conc_df.columns if 'Concentration Value' in col]]
        completing_concentration_values = (1 - concentration_values_isolated.sum(axis=1)).tolist()
        data.append(completing_concentration_values)
        columns.append('Component ' + str(final_component_index) + ' Concentration Value')

    component_conc_df = pd.DataFrame(data, columns).T
    
    component_conc_df.insert(loc=0, column = 'Total Sample Amount Unit', value = [total_sample_amount_unit]*n)
    component_conc_df.insert(loc=0, column = 'Total Sample Amount', value = [total_sample_amount]*n) # this needs to be added at the same amount

    return component_conc_df

In [195]:
concentration_df = concentration_from_linspace_all_info(plan, unity_filter=True)
concentration_df

,Total Sample Amount,Total Sample Amount Unit,Component 0 Name,Component 0 Concentration Unit,Component 0 Concentration Value,Component 1 Name,Component 1 Concentration Unit,Component 1 Concentration Value,Component 2 Name,Component 2 Concentration Unit,Component 2 Concentration Value,Component 3 Name,Component 3 Concentration Unit,Component 3 Concentration Value,Component 4 Name,Component 4 Concentration Unit,Component 4 Concentration Value
0,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0,ethanol,wtf,0,water,wtf,0.99989
1,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0,ethanol,wtf,0.0368421,water,wtf,0.963048
2,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0,ethanol,wtf,0.0736842,water,wtf,0.926206
3,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0,ethanol,wtf,0.110526,water,wtf,0.889364
4,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0,ethanol,wtf,0.147368,water,wtf,0.852522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0.01,ethanol,wtf,0.552632,water,wtf,0.437258
496,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0.01,ethanol,wtf,0.589474,water,wtf,0.400416
497,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0.01,ethanol,wtf,0.626316,water,wtf,0.363574
498,0.8,g,dppc,wtf,7.5e-05,dspepeg2000,wtf,3.5e-05,pfh,wtf,0.01,ethanol,wtf,0.663158,water,wtf,0.326732


In [204]:
def grab_single_component_concentration_info(component_conc_df, component_index):
    component_name_column = 'Component ' + str(component_index) + ' Name'
    component_unit_column = 'Component ' + str(component_index) + ' Concentration Unit'
    component_value_column = 'Component ' + str(component_index) + ' Concentration Value'
    
    component_name = component_conc_df[component_name_column]
    component_concentration = component_conc_df[component_unit_column]
    component_unit = component_conc_df[component_value_column]
    return component_name, component_concentration, component_unit

In [ ]:
https://stackoverflow.com/questions/30787901/how-to-get-a-value-from-a-pandas-dataframe-and-not-the-index-and-object-type

In [211]:
total_n_components = len([col for col in concentration_df.columns if 'Concentration Value' in col])
component_info = plan['Chemical Database']
for i, sample in concentration_df.iterrows():
    total_sample_amount = sample['Total Sample Amount']
    total_sample_amount_unit = sample['Total Sample Amount Unit']
#     # note iterate though what? like the length of components???
    for i in range(total_n_components): 
        component_name, component_concentration, component_unit = grab_single_component_concentration_info(concentration_df, i)
        mass = # yeah I think just go back to the original way of doing it, only because you know the entire column will be the same unit there is no other way todo it '
        # unless you are okay with making a list and then adding that to the existing or new df'

dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh

dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh
ethanol
water
dppc
dspepeg2000
pfh

In [71]:
def determine_component_mass(total_sample_amount, total_sample_amount_unit, component_values, component_unit, component_info):
    if total_sample_amount_unit == 'g' and component_unit == 'wtf':
        component_masses = total_sample_amount*component_values
        
    if total_sample_amount_unit == 'mL' and component_unit == 'mgpermL':
        component_masses = total_sample_amount*component_values/1000 # for now default mass = g, volume = mL

    if total_sample_amount_unit == 'mL' and component_unit == 'molarity':
        molecular_weight = component_info['Molecular Weight (g/mol)']
        component_masses = total_sample_amount*component_values*molecular_weight/1000
        
    return component_masses


In [8]:
def identify_unit(string):
    list_units = ['wtf','volf','molf','mgpermL','molarity']
    for unit in list_units:
        if unit in string:
            return unit

In [ ]:
# this method uses the whole column of the dictionary which still really defeats the purpose since we 
# we need to still iterate and call names and unit list and iterate side by sidew

component_info_dict = plan['Chemical Database']
component_names = plan['Component Shorthand Names']
component_units = plan['Component Concentration Units']
total_sample_amount_unit = plan['Sample Unit']
total_sample_amount = plan['Sample Amount']
component_info_dict['dppc']['Molecular Weight (g/mol)']

component_amounts_to_df = []
for i, (component_name, component_unit) in enumerate(zip(component_names, component_units)):
    if component_name in df_columns[i]: # This is to ensure correct order and names match up
        component_values = df[df_columns[i]]
        component_info = component_info_dict[component_name]
        if component_unit == 'wtf' or 'mgpermL' or 'molarity': # these are the unit that lead to mass outcomes
            component_masses = determine_component_mass(total_sample_amount, total_sample_amount_unit, component_values, component_unit, component_info)  
            df[component_name + ' amount mass g']  = component_masses
            
            